In [ ]:
#importing libraries
import pandas as pd
import pickle
from pandas import DataFrame, Series
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
import pybaseball as pyb
from pybaseball import statcast
import xgboost as xgb
import numpy as np
from scipy.stats import rankdata 
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
import matplotlib.lines 
from matplotlib.lines import Line2D

In [ ]:
#reading the data from Baseball Savant
df = statcast(start_dt='2022-04-01', end_dt='2022-10-5')

In [ ]:
#saving the data to a csv file
#Findins the columns in the data
pyb.cache.enable()

In [ ]:
#remove columns that are not needed
df = df[['game_date','player_name','p_throws','pitch_type','release_speed','release_spin_rate','pfx_x',
       'pfx_z', 'release_pos_x','release_pos_z','release_pos_y', 'release_extension','plate_x', 'plate_z','events',
       'description','batter','hit_distance_sc', 'launch_speed', 'launch_angle']]

In [ ]:
#order by release_speed
df = df.sort_values(by=['release_speed'], ascending=False)

In [ ]:
#insert VB AND HB
df.insert(5, 'Vertical Break', df['pfx_z']*12)
df.insert(6, 'Horizontal Break', df['pfx_x']*12)

In [ ]:
#Replace all the Description values of hit_into_play values with the Events values
df['description'] = np.where(df['description'] == 'hit_into_play', df['events'], df['description'])

In [ ]:
#add strikeouts and walks to the description values from events
df['description'] = np.where(df['events'] == 'strikeout', 'strikeout', df['description'])

In [ ]:
#remove events column
df = df.drop(['events'], axis=1)

In [ ]:
#remove pfx_x and pfx_z
df = df.drop(['pfx_x', 'pfx_z'], axis=1)

In [ ]:
#import razzball.csv from desktop to get the player id and name
df2 = pd.read_csv("/Users/johndavis/Desktop/razzball.csv")

In [ ]:
#rename MLBAMID to batter
df2 = df2.rename(columns={'MLBAMID': 'batter'})

In [ ]:
#Replace Name to Batter Name
df2 = df2.rename(columns={'Name': 'Batter Name'})

In [ ]:
#add a column to df called Batter Name which is the Batter Name from df2 that corresponds to the batter id from batter column in df
df = df.merge(df2[['batter', 'Batter Name']], on='batter', how='left')

In [ ]:
#create 2 new columns in df called Plate Side and Plate Height which are the plate_x and plate_z values * 12
df['Plate Side'] = df['plate_x']*12
df['Plate Height'] = df['plate_z']*12


In [ ]:
df = df[['game_date','player_name','p_throws','pitch_type','release_speed','Vertical Break','Horizontal Break','release_spin_rate','release_pos_x','release_pos_z','release_pos_y','release_extension','Plate Height','Plate Side','description','Batter Name','hit_distance_sc', 'launch_speed', 'launch_angle']]

In [ ]:
#change player name column to remove comma and have first name then last name instead of last name then first name
df['player_name'] = df['player_name'].str.replace(',', '')
df['player_name'] = df['player_name'].str.split().str[::-1].str.join(' ')

In [ ]:
#change batter name in df2 to Name 
df2 = df2.rename(columns={'Batter Name': 'Name'})

In [ ]:
#rename player_name to Pitcher Name
df = df.rename(columns={'player_name': 'Pitcher Name'})

In [ ]:
#put a new column next to Pitcher Name called Pitcher Team which is the Team Name from df2 that corresponds Pitcher Name from df
df = df.merge(df2[['Name', 'Team']], left_on='Pitcher Name', right_on='Name', how='left')
#put a new column next to Batter Name called Batter Team which is the Team Name from df2 that corresponds Batter Name from df
df = df.merge(df2[['Name', 'Team']], left_on='Batter Name', right_on='Name', how='left')

In [ ]:
df = df[['game_date','Pitcher Name','Team_x','p_throws','pitch_type','release_speed','Vertical Break','Horizontal Break','release_spin_rate','release_pos_x','release_pos_z','release_pos_y','release_extension','Plate Height','Plate Side','description','Batter Name','Team_y','hit_distance_sc', 'launch_speed', 'launch_angle']]

In [ ]:
#rename Team_x to Pitcher Team and Team_y to Batter Team
df = df.rename(columns={'Team_x': 'Pitcher Team', 'Team_y': 'Batter Team'})

In [ ]:
df.columns

In [ ]:
#create a new column called Ball in Play which gives a 1 if the description is either a field out, single, double, triple, or home run and a 0 if it is not
df['Ball in Play'] = np.where(df['description'].isin(['field_out', 'single', 'double', 'triple', 'home_run']), 1, 0)

In [ ]:
#create a new column called Strikeout which gives a 1 if the description is a strikeout and a 0 if it is not
df['Strikeout'] = np.where(df['description'] == 'strikeout', 1, 0)

In [ ]:
#create a new column called Hit which gives a 1 if the description is a hit and a 0 if it is not
df['Hit'] = np.where(df['description'].isin(['single', 'double', 'triple', 'home_run']), 1, 0)

In [ ]:
#create a new column called Walk which gives a 1 if the description is a walk and a 0 if it is not
df['Walk'] = np.where(df['description'] == 'walk', 1, 0)

In [ ]:
#create a new column called Hit by Pitch which gives a 1 if the description is a hit by pitch and a 0 if it is not
df['Hit by Pitch'] = np.where(df['description'] == 'hit_by_pitch', 1, 0)

In [ ]:
#create a new column called out which gives a 1 if the description is a field out or strikeout and a 0 if it is not
df['Out'] = np.where(df['description'].isin(['field_out', 'strikeout']), 1, 0)

In [ ]:
#list all possible description values
df['description'].unique()

In [ ]:
# assign run values to each description
#Set Run Value for each Description
home_run = 1.374328827219,
triple = 1.05755624961515,
double = 0.766083122898271,
single = 0.467292970729251
ball = 0.0636883289483747,
hit_by_pitch = 0.0636883289483747,
blocked_ball = 0.0636883289483747,
catcher_interf = 0.0636883289483747,
pitchout = 0.0636883289483747,
foul = -0.0380502742575014,
foul_tip = -0.0380502742575014,
foul_bunt = -0.0380502742575014,
bunt_foul_tip = -0.0380502742575014,
called_strike = -0.065092516089806,
swinging_strike = -0.118124935770601,
strikeout = -0.118124935770601,
swinging_strike_blocked = -0.118124935770601,
force_out = -0.1955687665555,
grounded_into_double_play = -0.1955687665555,
fielders_choice_out = -0.1955687665555,
fielders_choice = -0.1955687665555,
field_out = -0.1955687665555,
double_play = -0.1955687665555,
sac_fly = -0.236889645519856,
sac_bunt = -0.236889645519856,
field_error = -0.236889645519856,
catcher_interf = -0.789788814378052,
sac_fly_double_play = -0.789788814378052,
sac_bunt_double_play = -0.789788814378052,
triple_play = -0.789788814378052,


In [ ]:
#create a new column called Run Value which gives a run value for each description
df['Run Value'] = np.select([
    df['description'] == 'home_run',
    df['description'] == 'triple',
    df['description'] == 'double',
    df['description'] == 'single',
    df['description'] == 'ball',
    df['description'] == 'hit_by_pitch',
    df['description'] == 'blocked_ball',
    df['description'] == 'catcher_interf',
    df['description'] == 'pitchout',
    df['description'] == 'foul',
    df['description'] == 'foul_tip',
    df['description'] == 'foul_bunt',
    df['description'] == 'bunt_foul_tip',
    df['description'] == 'called_strike',
    df['description'] == 'swinging_strike',
    df['description'] == 'strikeout',
    df['description'] == 'swinging_strike_blocked',
    df['description'] == 'force_out',
    df['description'] == 'grounded_into_double_play',
    df['description'] == 'fielders_choice_out',
    df['description'] == 'fielders_choice',
    df['description'] == 'field_out',
    df['description'] == 'double_play',
    df['description'] == 'sac_fly',
    df['description'] == 'sac_bunt',
    df['description'] == 'field_error',
    df['description'] == 'catcher_interf',
    df['description'] == 'sac_fly_double_play',
    df['description'] == 'sac_bunt_double_play',
    df['description'] == 'triple_play'
], [
    home_run,
    triple,
    double,
    single,
    ball,
    hit_by_pitch,
    blocked_ball,
    catcher_interf,
    pitchout,
    foul,
    foul_tip,
    foul_bunt,
    bunt_foul_tip,
    called_strike,
    swinging_strike,
    strikeout,
    swinging_strike_blocked,
    force_out,
    grounded_into_double_play,
    fielders_choice_out,
    fielders_choice,
    field_out,
    double_play,
    sac_fly,
    sac_bunt,
    field_error,
    catcher_interf,
    sac_fly_double_play,
    sac_bunt_double_play,
    triple_play
], 0)


In [ ]:
#delete Ball in Play, Strikeout, Hit, Walk, Hit by Pitch, and Out columns
df = df.drop(['Ball in Play', 'Strikeout', 'Hit', 'Walk', 'Hit by Pitch', 'Out'], axis=1)

In [ ]:
#create a new dataframe df_avg which is the average of each pitch type release_speed, Vertical Break, Horizontal Break, release_spin_rate for each Pitcher Name with counts on every pitch type
df_avg = df.groupby(['Pitcher Name', 'pitch_type']).agg({'release_speed': 'mean', 'release_spin_rate': 'mean', 'Vertical Break': 'mean', 'Horizontal Break': 'mean', 'release_pos_x': 'mean', 'release_pos_z':'mean','release_extension':'mean'}).reset_index()
#create a new column called Pitch Count which is the count of each pitch type for each Pitcher Name
df_avg['Pitch Count'] = df.groupby(['Pitcher Name', 'pitch_type']).size().reset_index(name='Pitch Count')['Pitch Count']


In [ ]:
#order the Pitch Count column from highest to lowest
df_avg = df_avg.sort_values(by=['Pitcher Name', 'Pitch Count'], ascending=False)


In [ ]:
#reset index
df_avg = df_avg.reset_index(drop=True)

In [ ]:
#reorder columns so count is right of pitch type
df_avg = df_avg[['Pitcher Name', 'pitch_type','Pitch Count', 'release_speed', 'release_spin_rate', 'Vertical Break', 'Horizontal Break','release_pos_x','release_pos_z','release_extension']]

In [ ]:
#create a new column called Total Movement which is the sum of the absolute value of Vertical Break and Horizontal Break
df_avg['Total Movement'] = df_avg['Vertical Break'].abs() + df_avg['Horizontal Break']

In [ ]:
df_avg = df_avg[['Pitcher Name', 'pitch_type','Pitch Count', 'release_speed', 'release_spin_rate','Total Movement', 'Vertical Break', 'Horizontal Break','release_pos_x','release_pos_z','release_extension']]

In [ ]:
#switch horizontal break so that it is positive when the ball breaks away from the batter
df_avg['Horizontal Break'] = df_avg['Horizontal Break'] * -1

In [ ]:
#seperate into new dataframes for each pitcher
df_avg_Cole = df_avg[df_avg['Pitcher Name'] == 'Gerrit Cole']

In [ ]:
#set df_avg_pitcher to 2 decimal places
df_avg = df_avg.round(2)

In [ ]:
# create a dictionary of the pitch types and their corresponding colors AND make each color different 
pitch_colors = {'FF': '#FF0000', 'SL': 'blue', 'CH': '#FFFF00', 'CU': '#008000', 'FT': '#0000FF', 'SI': '#4B0082', 'FC': '#EE82EE', 'FS': '#FF0000', 'KC': '#FFA500', 'KN': '#FFFF00', 'EP': '#008000', 'SC': '#0000FF', 'FO': '#4B0082', 'PO': '#EE82EE', 'AB': '#FF0000', 'UN': '#FFA500', 'FA': '#FFFF00', 'IN': '#008000'}
pitch_types = ['FF', 'SL', 'CH', 'CU', 'FT', 'SI', 'FC', 'FS', 'KC', 'KN', 'EP', 'SC', 'FO', 'PO', 'AB', 'UN', 'FA', 'IN']


In [ ]:
#import basic stats csv file from desktop
df_stats = pd.read_csv("/Users/johndavis/Desktop/Basic_Stats.csv",low_memory=False, encoding='latin-1')

In [ ]:
#Remove the Ê character from the player names 
df_stats['Name'] = df_stats['Name'].str.replace('Ê', ' ')
#remove the * from the player names
df_stats['Name'] = df_stats['Name'].str.replace('*', ' ')

In [ ]:
#change Name to Pitcher Name
df_stats.rename(columns={'Name': 'Pitcher Name'}, inplace=True)

In [ ]:
# if name repeats add IP from that row to the row with the same name and take the average of the other stats 
df_stats = df_stats.groupby('Pitcher Name').agg({'IP': 'sum', 'ERA': 'mean', 'FIP': 'mean', 'Age': 'mean'}).reset_index()

In [ ]:
#using the same code as above, create an input box for the user to enter the name of the pitcher they want to see and create a plot for that pitcher
def pitcher_plot(pitcher_name):
    df_avg_pitcher = df_avg[df_avg['Pitcher Name'] == pitcher_name]
    fig, ax = plt.subplots(figsize=(12, 8))
    plt.style.use('fivethirtyeight')
    plt.scatter( df_avg_pitcher['Horizontal Break'], df_avg_pitcher['Vertical Break'], c=df_avg_pitcher['pitch_type'].map(pitch_colors), s=600, alpha=0.5)
    plt.axvline(x=0, color='black')
    plt.axhline(y=0, color='black')
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    plt.xticks(range(-20, 25, 5), labels=[f"{x}\"" for x in range(-20, 25, 5)])
    plt.yticks(range(-20, 25, 5), labels=[f"{x}\"" for x in range(-20, 25, 5)])
    plt.xlabel('Horizontal Break (inches)')
    plt.ylabel('Vertical Break (inches)')
    plt.title(pitcher_name + ' Pitch Movement')
    #put the two letter pitch type in the middle of each dot on the plot
    for i, txt in enumerate(df_avg_pitcher['pitch_type']):
        ax.annotate(txt, (df_avg_pitcher['Horizontal Break'].iloc[i], df_avg_pitcher['Vertical Break'].iloc[i]), ha='center', va='center')
    #create a legend in the bottom right corner of the plot which has the average release speed, spin rate, and total movement for each pitch type
    for pitch_type in pitch_types:
        if pitch_type in df_avg_pitcher['pitch_type'].values:
            plt.scatter([], [], c=pitch_colors[pitch_type], label=pitch_type + ' ' + str(df_avg_pitcher[df_avg_pitcher['pitch_type'] == pitch_type]['release_speed'].values[0]) + ' mph ')
    plt.legend(scatterpoints=1, frameon=False, labelspacing=1, loc='lower right')
    #change font to black
    plt.rcParams['text.color'] = 'black'
    plt.rcParams['axes.labelcolor'] = 'black'
    plt.rcParams['xtick.color'] = 'black'
    plt.rcParams['ytick.color'] = 'black'
    #change the y axis numbers to black font
    ax.tick_params(axis='y', colors='black')
    #remove the basic stats from the plot
    plt.rcParams['axes.facecolor'] = 'white'
    plt.show()


In [ ]:
#make a function that will create a pie chart for the user to see the percentage of each pitch type for a given pitcher
def pitcher_pie(pitcher_name):
    df_avg_pitcher = df_avg[df_avg['Pitcher Name'] == pitcher_name]
    fig, ax = plt.subplots(figsize=(12, 8))
    plt.style.use('fivethirtyeight')
    #create a list of the pitch types and a list of the percentage of pitches that are that pitch type
    pitch_types = df_avg_pitcher['pitch_type'].values
    pitch_percent = df_avg_pitcher['Pitch Count'].values / df_avg_pitcher['Pitch Count'].sum() * 100
    #create the pie chart
    plt.pie(pitch_percent, labels=pitch_types, colors=[pitch_colors[x] for x in pitch_types], autopct='%1.1f%%', startangle=90)
    plt.title(pitcher_name + ' Pitch Distribution')
   #never repeat the same color in the pie chart
    plt.rcParams['axes.prop_cycle'] = plt.cycler(color=pitch_colors.values())
    #make FS and FF different colors
    plt.rcParams['axes.prop_cycle']
    plt.show()
        

In [ ]:
#import basic stats csv file from desktop
df_stats = pd.read_csv("/Users/johndavis/Desktop/Basic_Stats.csv",low_memory=False, encoding='latin-1')

In [ ]:
#Remove the Ê character from the player names 
df_stats['Name'] = df_stats['Name'].str.replace('Ê', ' ')

In [ ]:
#remove the * from the player names
df_stats['Name'] = df_stats['Name'].str.replace('*', ' ')

In [ ]:
#change Name to Pitcher Name
df_stats.rename(columns={'Name': 'Pitcher Name'}, inplace=True)

In [ ]:
# if name repeats add IP from that row to the row with the same name and take the average of the other stats 
df_stats = df_stats.groupby('Pitcher Name').agg({'IP': 'sum', 'ERA': 'mean', 'FIP': 'mean', 'Age': 'mean'}).reset_index()


In [ ]:
#reorder the columns so age is right of the pitcher name
df_stats = df_stats[['Pitcher Name', 'Age', 'IP', 'ERA', 'FIP']]

In [ ]:
#create a function that displays a visual table of the stats for a given pitcher
def pitcher_stats(pitcher_name):
    df_stats_avg = df_stats[df_stats['Pitcher Name'] == pitcher_name]
    fig, ax = plt.subplots(figsize=(12, 8))
    plt.style.use('fivethirtyeight')
    #create a table with the stats for the given pitcher
    table = plt.table(cellText=df_stats_avg.values, colLabels=df_stats_avg.columns, loc='center')
    table.set_fontsize(14)
    table.scale(1, 2)
    ax.axis('off')
    #order the visual so that age is first, then IP, then ERA, then FIP
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    

    plt.title(pitcher_name + ' Stats')
    plt.show()

    

In [ ]:
# create a function that will plot release_pos_x and release_pos_z for a given pitcher for each pitch type
def pitcher_release(pitcher_name):
    df_avg_pitcher = df_avg[df_avg['Pitcher Name'] == pitcher_name]
    fig, ax = plt.subplots(figsize=(12, 8))
    plt.style.use('fivethirtyeight')
    #create a scatter plot for each pitch type
    for pitch_type in pitch_types:
        if pitch_type in df_avg_pitcher['pitch_type'].values:
            plt.scatter(df_avg_pitcher[df_avg_pitcher['pitch_type'] == pitch_type]['release_pos_x'], df_avg_pitcher[df_avg_pitcher['pitch_type'] == pitch_type]['release_pos_z'], c=pitch_colors[pitch_type], s=600, alpha=0.5, label=pitch_type)
    plt.axvline(x=0, color='black')
    plt.axhline(y=0, color='black')
    plt.xlim(-3, 3)
    plt.ylim(-3, 3)
    plt.xlabel('Horizontal Release (feet)')
    plt.ylabel('Vertical Release (feet)')
    plt.title(pitcher_name + ' Release Position')
    plt.legend(scatterpoints=1, frameon=False, labelspacing=1, loc='lower right')
    #change font to black
    plt.rcParams['text.color'] = 'black'
    plt.rcParams['axes.labelcolor'] = 'black'
    plt.rcParams['xtick.color'] = 'black'
    plt.rcParams['ytick.color'] = 'black'
    #change the y axis numbers to black font
    ax.tick_params(axis='y', colors='black')
    plt.show()

In [ ]:
#create a function with a visual table of pitch type, release speed, spin rate, Vertical Break, Horizontal Break, for a given pitcher do not include total movement
def pitcher_table(pitcher_name):
    df_avg_pitcher = df_avg[df_avg['Pitcher Name'] == pitcher_name]
    fig, ax = plt.subplots(figsize=(12, 8))
    plt.style.use('fivethirtyeight')
    #create a table with the stats for the given pitcher
    table = plt.table(cellText=df_avg_pitcher[['pitch_type', 'release_speed', 'release_spin_rate', 'Vertical Break', 'Horizontal Break']].values, colLabels=df_avg_pitcher[['pitch_type', 'release_speed', 'release_spin_rate', 'Vertical Break', 'Horizontal Break']].columns, loc='center')
    table.set_fontsize(14)
    table.scale(1, 2)
    ax.axis('off')
    plt.title(pitcher_name + ' Pitch Stats')
    plt.show()

In [ ]:
#create a function called total_pitcher which combines the pitcher_plot, pitcher_pie, pitcher_stats,and pitcher_table functions
def total_pitcher(pitcher_name):
    pitcher_stats(pitcher_name)
    pitcher_plot(pitcher_name)
    pitcher_pie(pitcher_name)
    pitcher_table(pitcher_name)

In [ ]:
total_pitcher('Gerrit Cole')